### Check valid parentheses
+ parentheses only: a stack
+ parentheses, square brackets, curly brackets: a stack + matching dict e.g. leetcode 20

In [39]:
s1 = "()[{}]"
s2 = "([{}]"
s3 = "([)]"

def f(s):
    stack, D = [], {"(":")", "[":"]", "{":"}"}
    for c in s:
        if c in D:
            stack.append(c)
        elif stack==[] or D[stack[-1]]!=c:
            return False
        else:
            stack.pop()
    return len(stack)==0

print( f(s1), f(s2), f(s3) )

True False False


### Catalan number: C(n) = (2n  n)/(n+1)
+ n-number stack permutation
+ n-node binary search trees e.g. leetcode 95, 96

C(1)=1, C(2)=2, C(3)=5, C(4)=14

In [56]:
L = [1,2,3] # 5 answers exclude [1,3,2]

ans = []
def stackPermutation(L, S=[], out=[]):
    if not L and not S:
        ans.append(out)
    if L:
        stackPermutation( L[:-1], S+[L[-1]], out )
    if S:
        stackPermutation( L, S[:-1], out+[S[-1]] )
        
stackPermutation(L)
print( ans )

[[1, 2, 3], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]


In [65]:
class TreeNode:
    def __init__(self, val, left, right):
        self.val, self.left, self.right = val, left, right

def genTrees(L):            
    if L==[]:
        return [None]
    else:
        forest = []
        for i in range(len(L)): # be root alternatively
            lForest = genTrees( L[:i] )
            rForest = genTrees( L[i+1:] )
            for lTree in lForest:
                for rTree in rForest:
                    forest.append( TreeNode(L[i], lTree, rTree) )
        return forest

forest = genTrees([1,2,3])
dlr    = lambda T: [T.val] + dlr(T.left) + dlr(T.right) if T else []
ldr    = lambda T: ldr(T.left) + [T.val] + ldr(T.right) if T else []
for T in forest:
    print( dlr(T), ldr(T) )

[1, 2, 3] [1, 2, 3]
[1, 3, 2] [1, 2, 3]
[2, 1, 3] [1, 2, 3]
[3, 1, 2] [1, 2, 3]
[3, 2, 1] [1, 2, 3]


### Expressions evaluation
+ Split
+ Change order:
    + infix to prefix: need 2 stacks
    + infix to postfix: need 1 stack (BEST)
+ Compute

e.g. leetcode 150. Evaluate Reverse Polish Notation

e.g. leetcode 227. Basic Calculator II

In [83]:
s1, s2 = "1/(2*3-(4+5))*(6-7)", "10*(5+28-3)/6^2"
print( eval(s1), eval(s2.replace("^","**")) )

0.3333333333333333 8.333333333333334


In [84]:
# stage 1: split
def split(s):
    infix, O = [], {"+":1, "-":1, "*":2, "/":2, "^":3, "(":10, ")":-10}
    for c in s.replace(" ",""):
        if c in O:
            if c=="-" and (not infix or infix[-1]=="("):
                infix += ["0", "-"]
            else:
                infix.append(c)
        else:
            if not infix or infix[-1] in O:
                infix.append(c)
            else:
                infix[-1]+=c
    return infix

print( split(s1) )
print( split(s2) )

['1', '/', '(', '2', '*', '3', '-', '(', '4', '+', '5', ')', ')', '*', '(', '6', '-', '7', ')']
['10', '*', '(', '5', '+', '28', '-', '3', ')', '/', '6', '^', '2']


In [87]:
# stage 2: infix to postfix
inf1 = ['1', '/', '(', '2', '*', '3', '-', '(', '4', '+', '5', ')', ')', '*', '(', '6', '-', '7', ')']
ans1 = ['1', '2', '3', '*', '4', '5', '+', '-', '/', '6', '7', '-', '*']
inf2 = ['10', '*', '(', '5', '+', '28', '-', '3', ')', '/', '6', '^', '2']
ans2 = ['10', '5', '28', '+', '3', '-', '*', '6', '2', '^', '/']

def infix2postfix(inf): # stack for operator only # ans for all without "(" and ")"
    operatorD, stack, ans = {"+":1, "-":1, "*":2, "/":2, "^":3, "(":10, ")":-10}, [], [] # "(" is highest out of stack but lowest in the stack
    for c in inf:
        if c in operatorD:
            while stack and (stack[-1]!="(") and operatorD[c] <= operatorD[stack[-1]]: # no pop if either: empty, top=="(", larger priority
                ans.append(stack.pop())
            if c!=")":
                stack.append(c)
            else:
                stack.pop() # pop "("
        else: # operand
            ans.append(c)
    return ans+stack[::-1]

print( infix2postfix(inf1) == ans1 )
print( infix2postfix(inf2) == ans2 )

True
True


In [111]:
# stage 3: compute
post1 = ['1', '2', '3', '*', '4', '5', '+', '-', '/', '6', '7', '-', '*']
ans1  = 0.333
post2 = ['10', '5', '28', '+', '3', '-', '*', '6', '2', '^', '/']
ans2  = 8.333

def compute(post): # stack for operand only
    stack, ops = [], {"+":lambda a,b:a+b, "-":lambda a,b:a-b, "*":lambda a,b:a*b, "/":lambda a,b:a/b, "^":lambda a,b:a**b}
    for c in post:
        if c not in ops:
            stack.append( float(c) )
        else:
            x = stack.pop()
            y = stack.pop()
            stack.append( ops[c](y,x) ) # because x-y -> xy-
    return stack.pop()

print( compute(post1), compute(post2) )

0.3333333333333333 8.333333333333334


### Next greater element
e.g. leetcode 739. Daily Temperatures

In [2]:
L = [73,74,75,71,69,72,76,73]

stack = [ (0,L[0]) ]             # (i,L[i])
ans   = [ (None,None) ] * len(L) # next greater element: (index,element)
for i in range(1,len(L)):
    while stack and stack[-1][1]<L[i]: # compare to infix2postfix # same: while stack; different: pop while self is larger
        idx, _   = stack.pop()
        ans[idx] = (i,L[i])
    stack.append( (i,L[i]) )
while stack:
    idx, _   = stack.pop()
    ans[idx] = (-1,None)
print( ans )

[(1, 74), (2, 75), (6, 76), (5, 72), (5, 72), (6, 76), (-1, None), (-1, None)]
